In [ ]:
import sys
sys.path.append('/Users/vdk/Software/code/')
import h5py
import pandas as pd
from astropy.table import Table, vstack
import numpy as np
from scipy.interpolate import interp1d
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt
import glob
from python_modules.muonpipe import usefull_func
import seaborn as sns
from ctapipe.image import tailcuts_clean
import scipy.stats as stats
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
    'k'
]
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

start_date_2020 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
filename = '/Users/vdk/muons2024/lapalma_simulations/simtel/zenith10/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry


ring_completeness_cut = 0.6
ring_containment_cut = 0.99

## Data loading

In [ ]:
fits_file = '/Users/vdk/muons2024/data/20240310fits/muons_LST-1.Run17043.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data


event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 

filename_event_ids = '/Users/vdk/muons2024/data/17043_muons_08complet.txt'
#np.save(filename_event_ids, event_ids)

df_real_data_17043 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

### Proper image extractor

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/data/17043/global_peak_muon_images/muon_images/*')
data_dl1_images = []
for file in image_folder:
    tmp_result = np.loadtxt(file)
    data_dl1_images.append(tmp_result)

#### Run 2973 as in performance paper

In [ ]:
fits_files = glob.glob('/Users/vdk/muons2024/performance_paper/run2973/*')

dat = Table.read(fits_files[0], format='fits')

for filename in fits_files:
    dat2 = Table.read(filename, format='fits')
    dat = vstack([dat, dat2])


df = dat.to_pandas()    
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.6
ring_containment_cut = 0.99

#np.save(filename_event_ids, event_ids)

df_real_data_2973 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

df_real_data_2973

## Simulation loading

### GlobalPeakWindowSum standard config

In [ ]:
global_fits_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/tables/muon_table_run1.fits'
global_dl1_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/dl1_run101_muon.h5'

dat = Table.read(global_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]


event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

global_sim_dl1_image = []
global_sim_dl1_peak_time = []
global_sim_dl1_image_mask = []
with h5py.File(global_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            global_sim_dl1_image.append(event_record[2])
            global_sim_dl1_peak_time.append(event_record[3])
            global_sim_dl1_image_mask.append(event_record[4])
        if len(global_sim_dl1_image) == 542:
            break


### Ruben config

In [ ]:
official_fits_file = '/Users/vdk/muons2024/config_comparison/larger_mirror_degraded_reflection/ruben_config/tables/muon_table_run1_tailcuts63.fits'
official_dl1_file = '/Users/vdk/muons2024/config_comparison/larger_mirror_degraded_reflection/ruben_config/dl1_run101_muon_tailcuts63.h5'

dat = Table.read(official_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_official = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

official_sim_dl1_image = []
official_sim_dl1_peak_time = []
official_sim_dl1_image_mask = []
with h5py.File(official_dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            official_sim_dl1_image.append(event_record[2])
            official_sim_dl1_peak_time.append(event_record[3])
            official_sim_dl1_image_mask.append(event_record[4])
        if len(official_sim_dl1_image) > 542:
            break


### Rubens config but tailcuts 8/4

In [ ]:
official_fits_file = '/Users/vdk/muons2024/config_comparison/ruben_config/tables/muon_table_run1_tailcuts84.fits'
#official_dl1_file = '/Users/vdk/muons2024/config_comparison/ruben_config/dl1_run101_muon_tailcuts63.h5'

dat = Table.read(official_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_official_tailcuts84 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]


### Georgios muons + standard lstchain config

In [ ]:
official_fits_file = '/Users/vdk/muons2024/config_comparison/georgios_muons_standard_config/tables/*'
#official_dl1_file = '/Users/vdk/muons2024/config_comparison/ruben_config/dl1_run101_muon_tailcuts63.h5'

listdir= glob.glob('/Users/vdk/muons2024/georgios_muons/output/table_muons/*')
dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    dat = vstack([dat, dat2])

        
df = dat.to_pandas()
df_sim_data_georgios_standard = df[df['good_ring'] & (df['muon_efficiency'] < 1) & (df['ring_containment'] >  ring_containment_cut) & (df_good_data['ring_completeness'] > ring_completeness_cut) & (df_good_data['size_outside'] < 500)]



### Standard config + integration correction without NSB tuning

In [ ]:
global_cor_fits_file = '/Users/vdk/muons2024/config_comparison/integration_correction_no_nsb_tune/muon_table_run1.fits'
global_cor_dl1_file = '/Users/vdk/muons2024/config_comparison/integration_correction_no_nsb_tune/dl1_run101_muon.h5'

dat = Table.read(global_cor_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global_integration_cor = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

global_cor_sim_dl1_image = []
global_cor_sim_dl1_peak_time = []
global_cor_sim_dl1_image_mask = []
with h5py.File(global_cor_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            global_cor_sim_dl1_image.append(event_record[2])
            global_cor_sim_dl1_peak_time.append(event_record[3])
            global_cor_sim_dl1_image_mask.append(event_record[4])
        if len(global_cor_sim_dl1_image) == 542:
            break


### Standard config + integration correction + NSB tuning

In [ ]:
global_cor_nsbtune_fits_file = '/Users/vdk/muons2024/config_comparison/integration_correction/triple_tuning/muon_table_run1_nsb_tune.fits'
global_cor_nsbtune_dl1_file = '/Users/vdk/muons2024/config_comparison/integration_correction/triple_tuning/dl1_run101_muon.h5'

dat = Table.read(global_cor_nsbtune_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

ring_completeness_cut = 0
ring_containment_cut = 0

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global_integration_cor_nsbtune = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

global_cor_nsbtune_sim_dl1_image = []
global_cor_nsbtune_sim_dl1_peak_time = []
global_cor_nsbtune_sim_dl1_image_mask = []
with h5py.File(global_cor_nsbtune_dl1_file, 'r') as f:
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    data = dataset[:]
    
    for event_record in data:
        if event_record[1] in event_ids:
            global_cor_nsbtune_sim_dl1_image.append(event_record[2])
            global_cor_nsbtune_sim_dl1_peak_time.append(event_record[3])
            global_cor_nsbtune_sim_dl1_image_mask.append(event_record[4])
        #if len(global_cor_nsbtune_sim_dl1_image) == 542:
        #    break


df_sim_data_global_integration_cor_nsbtune

### Standard config + integration correction + NSB tuning + tuned reflection 94% from nominal

In [ ]:
global_cor_nsbtune_fits_file = '/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/muon_table_run1_nsb_tune.fits'
global_cor_nsbtune_dl1_file = '/Users/vdk/muons2024/config_comparison/integration_correction/triple_tuning/dl1_run101_muon.h5'

dat = Table.read(global_cor_nsbtune_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global_integration_cor_nsbtune_ref0736 = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

# global_cor_nsbtune_sim_dl1_image = []
# global_cor_nsbtune_sim_dl1_peak_time = []
# global_cor_nsbtune_sim_dl1_image_mask = []
# with h5py.File(global_cor_nsbtune_dl1_file, 'r') as f:
#     dataset = f['dl1/event/telescope/image/LST_LSTCam']
#     data = dataset[:]
    
#     for event_record in data:
#         if event_record[1] in event_ids:
#             global_cor_nsbtune_sim_dl1_image.append(event_record[2])
#             global_cor_nsbtune_sim_dl1_peak_time.append(event_record[3])
#             global_cor_nsbtune_sim_dl1_image_mask.append(event_record[4])
#         #if len(global_cor_nsbtune_sim_dl1_image) == 542:
        #    break


df_sim_data_global_integration_cor_nsbtune_ref0736

### Standard config + integration correction + NSB tuning + 0.125R ring size

In [ ]:
global_cor_nsbtune_0125R_fits_file = '/Users/vdk/muons2024/config_comparison/integration_correction/nsb_tuned/0125R_ringsize/muon_table_run1.fits'
global_cor_dl1_file = '/Users/vdk/muons2024/config_comparison/integration_correction/nsb_tuned/dl1_run101_muon.h5'

dat = Table.read(global_cor_nsbtune_0125R_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global_integration_cor_nsbtune_0125R = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

# global_cor_sim_dl1_image = []
# global_cor_sim_dl1_peak_time = []
# global_cor_sim_dl1_image_mask = []
# with h5py.File(global_cor_dl1_file, 'r') as f:
#     dataset = f['dl1/event/telescope/image/LST_LSTCam']
#     data = dataset[:]
    
#     for event_record in data:
#         if event_record[1] in event_ids:
#             global_cor_sim_dl1_image.append(event_record[2])
#             global_cor_sim_dl1_peak_time.append(event_record[3])
#             global_cor_sim_dl1_image_mask.append(event_record[4])
#         if len(global_cor_sim_dl1_image) == 542:
#             break


### Standard config + integration correction + NSB tuning + 0.4R ring size

In [ ]:
global_cor_nsbtune_0125R_fits_file = '/Users/vdk/muons2024/config_comparison/integration_correction/nsb_tuned/04R_ringsize/muon_table_run1.fits'
global_cor_dl1_file = '/Users/vdk/muons2024/config_comparison/integration_correction/nsb_tuned/dl1_run101_muon.h5'

dat = Table.read(global_cor_nsbtune_0125R_fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                            (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]))

df_sim_data_global_integration_cor_nsbtune_04R = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut) &
                                           (df_good_data['impact_parameter'] < max_impact) &
                                           (df_good_data['impact_parameter'] > min_impact)]

# global_cor_sim_dl1_image = []
# global_cor_sim_dl1_peak_time = []
# global_cor_sim_dl1_image_mask = []
# with h5py.File(global_cor_dl1_file, 'r') as f:
#     dataset = f['dl1/event/telescope/image/LST_LSTCam']
#     data = dataset[:]
    
#     for event_record in data:
#         if event_record[1] in event_ids:
#             global_cor_sim_dl1_image.append(event_record[2])
#             global_cor_sim_dl1_peak_time.append(event_record[3])
#             global_cor_sim_dl1_image_mask.append(event_record[4])
#         if len(global_cor_sim_dl1_image) == 542:
#             break


## Picking up the mask

#### Additional cleaning for proper muon tailcut parameters

In [ ]:
new_data_mask = []
for image in data_dl1_images:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_mask.append(mask)
    

new_sim_mask_global = []
for image in global_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask_global.append(mask)
    

new_sim_mask_global_cor = []
for image in global_cor_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask_global_cor.append(mask)

    
new_sim_official_mask = []
for image in official_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_official_mask.append(mask)


new_sim_mask_global_cor_nsbtune = []
for image in global_cor_nsbtune_sim_dl1_image:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask_global_cor_nsbtune.append(mask)
    


In [ ]:
flattened_sim_official = np.array(official_sim_dl1_image).flatten()
flattened_sim_global = np.array(global_sim_dl1_image).flatten()
flattened_sim_global_cor = np.array(global_cor_sim_dl1_image).flatten()
flattened_sim_global_cor_nsbtune = np.array(global_cor_nsbtune_sim_dl1_image).flatten()
flattened_data = np.array(data_dl1_images).flatten()


official_sim_masks_flat = np.array(official_sim_dl1_image_mask).flatten()
global_sim_masks_flat = np.array(global_sim_dl1_image_mask).flatten()

official_sim_masks_flat = np.array(new_sim_official_mask).flatten()
global_sim_masks_flat = np.array(new_sim_mask_global).flatten()
data_masks_flat = np.array(new_data_mask).flatten()
global_cor_sim_masks_flat = np.array(new_sim_mask_global_cor).flatten()
global_cor_nsbtune_sim_masks_flat = np.array(new_sim_mask_global_cor_nsbtune).flatten()

# Separate pixel values based on the mask
data_pixels_with_mask_1 = flattened_data[data_masks_flat == 1]
data_pixels_with_mask_0 = flattened_data[data_masks_flat == 0]

official_sim_pixels_with_mask_1 = flattened_sim_official[official_sim_masks_flat == 1]
official_sim_pixels_with_mask_0 = flattened_sim_official[official_sim_masks_flat == 0]

global_sim_pixels_with_mask_1 = flattened_sim_global[global_sim_masks_flat == 1]
global_sim_pixels_with_mask_0 = flattened_sim_global[global_sim_masks_flat == 0]

global_cor_sim_pixels_with_mask_1 = flattened_sim_global_cor[global_cor_sim_masks_flat == 1]
global_cor_sim_pixels_with_mask_0 = flattened_sim_global_cor[global_cor_sim_masks_flat == 0]

global_cor_nsbtune_sim_pixels_with_mask_1 = flattened_sim_global_cor_nsbtune[global_cor_nsbtune_sim_masks_flat == 1]
global_cor_nsbtune_sim_pixels_with_mask_0 = flattened_sim_global_cor_nsbtune[global_cor_nsbtune_sim_masks_flat == 0]
   

In [ ]:
plt.figure(figsize=(10,10))

usefull_func.hist_wo_outliers(
    data_pixels_with_mask_0[:1855000], 
    label = 'Run17043', 
    density=True, 
    show_outliers=False, 
    outlier_value=30, 
    bins = 300, 
    color = 'lightblue', 
    alpha = 1)

# usefull_func.hist_wo_outliers(
#     official_sim_pixels_with_mask_0, 
#     label = 'Ruben config', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bins = 1855, 
#     color = colors[6], 
#     alpha = 0.5
# )

# usefull_func.hist_wo_outliers(
#     global_sim_pixels_with_mask_0, 
#     label = 'Standard Config', 
#     density=True, 
#     mean_line=False, 
#     show_outliers=True, 
#     outlier_value=30, 
#     bins = 300, 
#     color = colors[5], 
#     alpha = 1,
#     histtype = 'step', linewidth=2
# )

usefull_func.hist_wo_outliers(
    global_cor_sim_pixels_with_mask_0[:1855000], 
    label = 'Standard Config Simulation', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 300, 
    color = colors[2], 
    alpha = 0.7,
    histtype = 'step', linewidth=3
)

usefull_func.hist_wo_outliers(
    global_cor_nsbtune_sim_pixels_with_mask_0[:1855000], 
    label = 'Standard Config Simulation + NSB tune to Run17043', 
    density=True, 
    mean_line=False, 
    show_outliers=True, 
    outlier_value=30, 
    bins = 300, 
    color = colors[1], 
    alpha = 0.7,
    histtype = 'step', linewidth=3
)

plt.xlabel('Charge per pixel', fontsize = 14)
plt.ylabel('Probability density', fontsize = 14)
plt.ylim(0,0.35)
plt.xlim(-6,15)
#plt.title("Charge per pixel for pixels without signal")
plt.legend(prop={'size': 13})
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/dl1_hist_pixel_charge_no_signal_nsbtuned_clean.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:

mask = tailcuts_clean(camgeom, data_dl1_images[2], picture_thresh=10, boundary_thresh=5)
fig, ax = plt.subplots(1, 2, figsize=(18, 10))
disp = CameraDisplay(
    camgeom, image=data_dl1_images[2], cmap="gray", ax = ax[0], title="Pixels with signal"
)
disp.highlight_pixels(mask, alpha=0.8, linewidth=2, color="green")

### Data for all years

In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
print(sorted(muon_files))

df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[2:]] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data_real = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['size_outside'] < 500)]
del(df_all_data)
df_good_data_real

### Ring size vs radius

In [ ]:
bin_number = 8
alpha_reg = 0.7
size_outside = 500

plt.figure(figsize = (11,11))

# sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real['ring_containment'] > ring_containment_cut) & 
#                                                 (df_good_data_real['event_time'] > start_date_2021) &
#                                                 (df_good_data_real['event_time'] < end_date_2021)], 
#             y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
#                                                (df_good_data_real['ring_containment'] > ring_containment_cut) &
#                                                 (df_good_data_real['event_time'] > start_date_2021) &
#                                                 (df_good_data_real['event_time'] < end_date_2021)], 
#             x_bins=bin_number, color=colors[0], label = 'Data for 2021 year')

# sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real['ring_containment'] > ring_containment_cut) & 
#                                                 (df_good_data_real['event_time'] > start_date_2022) &
#                                                 (df_good_data_real['event_time'] < end_date_2022)], 
#             y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
#                                                (df_good_data_real['ring_containment'] > ring_containment_cut) &
#                                                 (df_good_data_real['event_time'] > start_date_2022) &
#                                                 (df_good_data_real['event_time'] < end_date_2022)], 
#             x_bins=bin_number, color=colors[1], label = 'Data for 2022 year')

# sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real['ring_containment'] > ring_containment_cut) & 
#                                                 (df_good_data_real['event_time'] > start_date_2023) &
#                                                 (df_good_data_real['event_time'] < end_date_2023)], 
#             y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
#                                                (df_good_data_real['ring_containment'] > ring_containment_cut) &  
#                                                 (df_good_data_real['event_time'] > start_date_2023) &
#                                                 (df_good_data_real['event_time'] < end_date_2023)], 
#             x_bins=bin_number, color=colors[2], label = 'Data for 2023 year')

# sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
#                                                  (df_good_data_real['ring_containment'] > ring_containment_cut) & 
#                                                 (df_good_data_real['event_time'] > start_date_2024) &
#                                                 (df_good_data_real['event_time'] < end_date_2024)], 
#             y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
#                                                (df_good_data_real['ring_containment'] > ring_containment_cut) &  
#                                                 (df_good_data_real['event_time'] > start_date_2024) &
#                                                 (df_good_data_real['event_time'] < end_date_2024)], 
#             x_bins=bin_number, color=colors[1], label = 'Data for 2024 year')

sns.regplot(x = df_real_data['ring_radius'][(df_real_data['size_outside'] < size_outside)],
            y = df_real_data['ring_size'][(df_real_data['size_outside'] < size_outside)],
            x_bins=bin_number, color=colors[1], label = 'Run 17043')


#sns.regplot(x = df_sim_data_official['ring_radius'], y = df_sim_data_official['ring_size'], x_bins=bin_number, color='k', label = 'Ruben config', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_official_tailcuts84['ring_radius'], y = df_sim_data_official_tailcuts84['ring_size'], x_bins=bin_number, color='g', label = 'Ruben config tailcuts84', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_global['ring_radius'], y = df_sim_data_global['ring_size'], x_bins=bin_number, color=colors[4], label = 'Standard config', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_global_integration_cor['ring_radius'], y = df_sim_data_global_integration_cor['ring_size'], x_bins=bin_number, color=colors[3], label = 'Standard config + integration correction', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
sns.regplot(x = df_sim_data_global_integration_cor_nsbtune['ring_radius'], y = df_sim_data_global_integration_cor_nsbtune['ring_size'], x_bins=bin_number, color=colors[0], label = 'Standard config + integration correction + nsbtune', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_real_cut['ring_radius'], y = df_real_cut['ring_size'], x_bins=bin_number, color=colors[0], label = '2024 data', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_cut['ring_radius'], y = df_sim_cut['ring_size'], x_bins=bin_number, color=colors[1], label = 'Standard config + nsbtune', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_global_integration_cor_nsbtune_04R['ring_radius'], y = df_sim_data_global_integration_cor_nsbtune_04R['ring_size'], x_bins=bin_number, color=colors[2], label = 'Standard config + integration correction + nsbtune+0125R', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})

#sns.regplot(x = df_sim_data_globa_nsbtune['ring_radius'], y = df_sim_data_globa_nsbtune['ring_size'], x_bins=bin_number, color=colors[5], label = 'Standard Config + NSBTune + mirror_degraded_reflection 0.9016', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})
#sns.regplot(x = df_sim_data_georgios_standard['ring_radius'], y = df_sim_data_georgios_standard['ring_size'], x_bins=bin_number, color=colors[5], label = 'Georgios + Standard config', scatter_kws={'alpha': alpha_reg}, line_kws={'alpha': alpha_reg})

plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [pe]')
plt.grid(linestyle="dashed")
plt.ylim(1500,3000)
plt.xlim(0.95,1.25)
plt.legend(prop={'size': 10})
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/mcdata_size_2023-24year_allreflections.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
# ring_completeness_cut = 0.9
# ring_containment_cut = 0.9
df_real_data_cut = df_good_data_real[(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
                                     (df_good_data_real['ring_containment'] > ring_containment_cut) &  
                                     (df_good_data_real['event_time'] > start_date_2024) &
                                     (df_good_data_real['event_time'] < end_date_2024)
                                     ]

df_sim_cut = df_sim_data_global_integration_cor_nsbtune[
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > 0.95) & 
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < 1.25)
    ]
df_real_cut = df_real_data_cut[(df_real_data_cut['ring_radius'] > 0.95) & (df_real_data_cut['ring_radius'] < 1.25)]
#df_real_cut = df_real_data_2973[(df_real_data_2973['ring_radius'] > 0.95) & (df_real_data_2973['ring_radius'] < 1.25)]

data_mc = df_sim_cut

data_run = df_real_cut
# Define the bin edges
bins = np.linspace(0.95, 1.25, 8)  # Adjust the number of bins as needed

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Plotting
plt.figure(figsize=(10, 6))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='k', label='Standard config', s = 50)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label='2024 data', s = 50)

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.grid(alpha = 0.5)
plt.show()

In [ ]:
# df_real_data_cut = df_good_data_real[
#     (df_good_data_real['ring_completeness'] > ring_completeness_cut) &
#     (df_good_data_real['ring_containment'] > ring_containment_cut) &  
#     (df_good_data_real['ring_radius'] > 0.95) &  
#     (df_good_data_real['ring_radius'] < 1.25) &  
#     (df_good_data_real['event_time'] > start_date_2024) &
#     (df_good_data_real['event_time'] < end_date_2024)
# ]


df_real_data_cut = df_good_data_real[ 
    (df_good_data_real['ring_radius'] > 0.95) &  
    (df_good_data_real['ring_radius'] < 1.25) &  
    (df_good_data_real['event_time'] > start_date_2024) &
    (df_good_data_real['event_time'] < end_date_2024)
]

df_sim_cut = df_sim_data_global_integration_cor_nsbtune[
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > 0.95) & 
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < 1.25)
]

data_mc = df_sim_cut

data_run = df_real_data_cut
# Define the bins
bins = np.linspace(0.95, 1.25, 8)  # Adjust the number of bins as needed

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

binned_mc['ring_size'] = binned_mc['ring_size']

# Perform linear regression
mc_slope, mc_intercept = np.polyfit(binned_mc['ring_radius'], binned_mc['ring_size'], 1)
run_slope, run_intercept = np.polyfit(binned_run['ring_radius'], binned_run['ring_size'], 1)

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)

# Generate y values using the regression parameters
y_values_mc = mc_slope * x_values + mc_intercept
y_values_run = run_slope * x_values + run_intercept

# Plotting
plt.figure(figsize=(10, 6))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Standard config', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression')

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label='2024 data', s=50)
plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression')

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.grid(alpha=0.5)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Filter df_sim_data_global_integration_cor_nsbtune_ref0736
df_sim_data_global_integration_cor_nsbtune_ref0736_cut = df_sim_data_global_integration_cor_nsbtune_ref0736[
    (df_sim_data_global_integration_cor_nsbtune_ref0736['ring_radius'] > 0.95) & 
    (df_sim_data_global_integration_cor_nsbtune_ref0736['ring_radius'] < 1.25)
]

# Bin the data for df_sim_data_global_integration_cor_nsbtune_ref0736
df_sim_data_global_integration_cor_nsbtune_ref0736_cut['binned_radius'] = pd.cut(
    df_sim_data_global_integration_cor_nsbtune_ref0736_cut['ring_radius'], bins, include_lowest=True, labels=False
)
binned_sim_ref = df_sim_data_global_integration_cor_nsbtune_ref0736_cut.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_sim_ref['ring_size'] = binned_sim_ref['ring_size'] * 0.995
# Add a constant to the independent variable for statsmodels
binned_sim_ref['constant'] = 1

# Perform linear regression using statsmodels
model_sim_ref = sm.OLS(binned_sim_ref['ring_size'], binned_sim_ref[['constant', 'ring_radius']]).fit()

# Generate y values and confidence intervals using the regression models
y_values_sim_ref = model_sim_ref.predict(x_values_with_const)
prediction_sim_ref = model_sim_ref.get_prediction(x_values_with_const)
conf_int_sim_ref = prediction_sim_ref.conf_int()

# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation with tuned NSB to Run17043 (0.31 GHz)', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label='Data for 2024 year', s=50)
plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)

# Plotting Sim Ref Data
plt.scatter(binned_sim_ref['ring_radius'], binned_sim_ref['ring_size'], color='green', label='Simulation for 93% of nominal optical efficiency', s=50)
plt.plot(x_values, y_values_sim_ref, color='green', linestyle='--', label='93% of nominal optical efficiency')
plt.fill_between(x_values, conf_int_sim_ref[:, 0], conf_int_sim_ref[:, 1], color='green', alpha=0.2)

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.ylim(1800,2800)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/latest_config_size_vs_radius_2024year_reflection-7percent.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
df_real_data_cut = df_good_data_real[(df_good_data_real['ring_completeness'] > ring_completeness_cut) &
                                     (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                     (df_good_data_real['ring_radius'] > 0.95) & 
                                     (df_good_data_real['ring_radius'] < 1.25) &
                                     (df_good_data_real['event_time'] > start_date_2024) &
                                     (df_good_data_real['event_time'] < end_date_2024)
                                     ]

df_sim_cut = df_sim_data_global_integration_cor_nsbtune[
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > 0.95) & 
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < 1.25)
    ]
df_real_cut = df_real_data_cut[(df_real_data_cut['ring_radius'] > 0.95) & (df_real_data_cut['ring_radius'] < 1.25)]
#df_real_cut = df_real_data_2973[(df_real_data_2973['ring_radius'] > 0.95) & (df_real_data_2973['ring_radius'] < 1.25)]

data_mc = df_sim_cut

data_run = df_real_cut
# Define the bins
bins = np.linspace(0.95, 1.3, 8)  # Adjust the number of bins as needed

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

binned_mc['ring_size'] = binned_mc['ring_size']

# Add a constant to the independent variable for statsmodels
binned_mc['constant'] = 1
binned_run['constant'] = 1

# Perform linear regression using statsmodels
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()

y_values_run = model_run.predict(x_values_with_const)
prediction_run = model_run.get_prediction(x_values_with_const)
conf_int_run = prediction_run.conf_int()

# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation with tuned NSB to Run17043 (0.31 GHz)', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label='Data for 2024 year', s=50)
plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.ylim(1800,2800)
plt.grid(alpha=0.5)
#plt.show()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/latest_config_size_vs_radius_2024year_reflection_v2.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Define a function to perform the filtering and plotting for different cuts
def filter_and_plot(df_real_data, ring_completeness_cut, ring_containment_cut, color, linestyle):
    df_real_data_cut = df_real_data[
        (df_real_data['ring_completeness'] > ring_completeness_cut) &
        (df_real_data['ring_containment'] > ring_containment_cut) & 
        (df_real_data['ring_radius'] > 0.95) & 
        (df_real_data['ring_radius'] < 1.25)
    ]
    
    # Bin the data for Run
    df_real_data_cut['binned_radius'] = pd.cut(df_real_data_cut['ring_radius'], bins, include_lowest=True, labels=False)
    binned_run = df_real_data_cut.groupby('binned_radius').agg({
        'ring_radius': 'mean',
        'ring_size': 'mean'
    })

    # Perform linear regression using statsmodels
    binned_run['constant'] = 1
    model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()

    # Generate y values and confidence intervals using the regression models
    y_values_run = model_run.predict(x_values_with_const)
    prediction_run = model_run.get_prediction(x_values_with_const)
    conf_int_run = prediction_run.conf_int()

    # Plotting Data Run
    plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color=color, label=f'Completeness {ring_completeness_cut}, Containment {ring_containment_cut}', s=50)
    plt.plot(x_values, y_values_run, color=color, linestyle=linestyle)
    plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color=color, alpha=0.2)

# Define the bins
bins = np.linspace(0.95, 1.3, 8)  # Adjust the number of bins as needed



plt.figure(figsize=(12, 8))

# Colors and linestyles for different cuts
colors = ['orange', 'green', 'red', 'purple', 'brown']
linestyles = ['--', '-.', ':', '-', '--']

# Plot data for different cuts
cuts = [0.4, 0.5, 0.6, 0.7, 0.8]
for i, cut in enumerate(cuts):
    filter_and_plot(df_real_data, cut, cut, colors[i], linestyles[i])

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.ylim(1800, 2800)
plt.legend()
plt.grid(alpha=0.5)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Define a function to perform the filtering and plotting for a given year
def filter_and_plot(df_real_data, start_date, end_date, label, color):
    df_real_data_cut = df_real_data[
        (df_real_data['ring_completeness'] > ring_completeness_cut) &
        (df_real_data['ring_containment'] > ring_containment_cut) & 
        (df_real_data['ring_radius'] > 0.95) & 
        (df_real_data['ring_radius'] < 1.25) &
        (df_real_data['event_time'] > start_date) &
        (df_real_data['event_time'] < end_date)
    ]
    
    df_real_cut = df_real_data_cut[(df_real_data_cut['ring_radius'] > 0.95) & (df_real_data_cut['ring_radius'] < 1.25)]
    
    # Bin the data for Run
    df_real_cut['binned_radius'] = pd.cut(df_real_cut['ring_radius'], bins, include_lowest=True, labels=False)
    binned_run = df_real_cut.groupby('binned_radius').agg({
        'ring_radius': 'mean',
        'ring_size': 'mean'
    })

    # Perform linear regression using statsmodels
    binned_run['constant'] = 1
    model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()

    # Generate y values and confidence intervals using the regression models
    y_values_run = model_run.predict(x_values_with_const)
    prediction_run = model_run.get_prediction(x_values_with_const)
    conf_int_run = prediction_run.conf_int()

    # Plotting Data Run
    plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color=color, label=label, s=50)
    plt.plot(x_values, y_values_run, color=color, linestyle='--')
    plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color=color, alpha=0.2)

# Define the bins
bins = np.linspace(0.95, 1.3, 8)  # Adjust the number of bins as needed

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Perform linear regression using statsmodels
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()

# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation with tuned NSB', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# Plot data for each year
filter_and_plot(df_good_data_real, start_date_2024, end_date_2024, 'Data for 2024 year', 'orange')
filter_and_plot(df_good_data_real, start_date_2021, end_date_2021, 'Data for 2021 year', 'green')
filter_and_plot(df_good_data_real, start_date_2022, end_date_2022, 'Data for 2022 year', 'red')
filter_and_plot(df_good_data_real, start_date_2023, end_date_2023, 'Data for 2023 year', 'purple')

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.ylim(1800,2800)
plt.legend()
plt.grid(alpha=0.5)
plt.show()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/latest_config_size_vs_radius_all_years.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Define a function to perform the filtering and plotting for different cuts
def filter_and_plot(df_real_data, ring_completeness_cut, ring_containment_cut, color, linestyle):
    df_real_data_cut = df_real_data[
        (df_real_data['ring_completeness'] > ring_completeness_cut) &
        (df_real_data['ring_containment'] > ring_containment_cut) & 
        (df_real_data['ring_radius'] > 0.95) & 
        (df_real_data['ring_radius'] < 1.25)
    ]
    
    # Bin the data for Run
    df_real_data_cut['binned_radius'] = pd.cut(df_real_data_cut['ring_radius'], bins, include_lowest=True, labels=False)
    binned_run = df_real_data_cut.groupby('binned_radius').agg({
        'ring_radius': 'mean',
        'ring_size': 'mean'
    })

    # Perform linear regression using statsmodels
    binned_run['constant'] = 1
    model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()

    # Generate y values and confidence intervals using the regression models
    y_values_run = model_run.predict(x_values_with_const)
    prediction_run = model_run.get_prediction(x_values_with_const)
    conf_int_run = prediction_run.conf_int()

    # Plotting Data Run
    plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color=color, label=f'Completeness {ring_completeness_cut}, Containment {ring_containment_cut}', s=50)
    plt.plot(x_values, y_values_run, color=color, linestyle=linestyle)
    plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color=color, alpha=0.2)

# Define the bins
bins = np.linspace(0.95, 1.3, 8)  # Adjust the number of bins as needed





# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()


# Define start and end dates for filtering
start_date_2024 = '2024-01-01'
end_date_2024 = '2024-12-31'

# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data


# Colors and linestyles for different cuts
colors = ['orange', 'green', 'red', 'purple', 'brown']
linestyles = ['--', '-.', ':', '-', '--']

# Plot data for different cuts
for i, (ring_completeness_cut, ring_containment_cut) in enumerate(zip([0.4, 0.5, 0.6, 0.7, 0.8], [0, 0, 0, 0, 0])):
    filter_and_plot(df_good_data_real, ring_completeness_cut, ring_containment_cut, colors[i], linestyles[i])

# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.ylim(1800, 2800)
plt.legend()
plt.grid(alpha=0.5)
plt.show()

In [ ]:
df_good_data_real

In [ ]:
df_real_data_cut = df_real_data_17043[(df_real_data_17043['ring_completeness'] > ring_completeness_cut) &
                                     (df_real_data_17043['ring_containment'] > ring_containment_cut) &
                                     (df_real_data_17043['ring_radius'] > 0.95) & 
                                     (df_real_data_17043['ring_radius'] < 1.25)]

df_sim_cut = df_sim_data_global_integration_cor_nsbtune[
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] > 0.95) & 
    (df_sim_data_global_integration_cor_nsbtune['ring_radius'] < 1.25)
    ]

#df_real_cut = df_real_data_2973[(df_real_data_2973['ring_radius'] > 0.95) & (df_real_data_2973['ring_radius'] < 1.25)]

data_mc = df_sim_cut

data_run = df_real_data_cut
# Define the bins
bins = np.linspace(0.95, 1.3, 8)  # Adjust the number of bins as needed

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})

binned_mc['ring_size'] = binned_mc['ring_size']

# Add a constant to the independent variable for statsmodels
binned_mc['constant'] = 1
binned_run['constant'] = 1

# Perform linear regression using statsmodels
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Generate y values and confidence intervals using the regression models
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()

y_values_run = model_run.predict(x_values_with_const)
prediction_run = model_run.get_prediction(x_values_with_const)
conf_int_run = prediction_run.conf_int()

# Plotting
plt.figure(figsize=(12, 8))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation with tuned NSB to Run17043 (0.31 GHz)', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='Simulation regression line')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label='Data for 17043 run', s=50)
plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)


# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.ylim(1800,2800)
plt.grid(alpha=0.5)
plt.show()
#plt.savefig('/Users/vdk/muons2024/images/PlanPaper/latest_config_size_vs_radius_2024year_Run17043.pdf', dpi=200, format='pdf', bbox_inches='tight')

#### Run 2973 as in performance paper

In [ ]:
(binned_run['ring_size'] - binned_mc['ring_size'])/binned_run['ring_size']

In [ ]:

# Plotting
plt.figure(figsize=(10, 6))

# Plotting MC data
plt.errorbar(df_sim_data_globa_nsbtune['ring_radius'], df_sim_data_globa_nsbtune['ring_size'], 
             yerr=df_sim_data_globa_nsbtune['ring_size'] * 0.05, fmt='o', color='blue', label='MC')

# Plotting Data Run
plt.errorbar(df_real_data['ring_radius'], df_real_data['ring_size'], 
             yerr=df_real_data['ring_size'] * 0.05, fmt='o', color='black', label='Data (Run 2973)')

# Adding labels and legend
plt.xlabel('Muon ring radius [deg]')
plt.ylabel('Muon ring intensity [p.e.]')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
np.mean(df_real_data['ring_size'][(df_real_data['ring_radius'] > 0.94) & (df_real_data['ring_radius'] < 0.96)])

In [ ]:
np.mean(df_sim_data_globa_nsbtune['ring_size'][(df_sim_data_globa_nsbtune['ring_radius'] > 0.94) & (df_sim_data_globa_nsbtune['ring_radius'] < 0.96)])

In [ ]:
print(f"data {np.mean(df_real_data['ring_size'][(df_real_data['ring_radius'] > 1.01) & (df_real_data['ring_radius'] < 1.03)])}")
print(f"sim {np.mean(df_sim_data_globa_nsbtune['ring_size'][(df_sim_data_globa_nsbtune['ring_radius'] > 1.01) & (df_sim_data_globa_nsbtune['ring_radius'] < 1.03)])}")

In [ ]:
print(f"data {np.mean(df_real_data['ring_size'][(df_real_data['ring_radius'] > 1.06) & (df_real_data['ring_radius'] < 1.08)])}")
print(f"sim {np.mean(df_sim_data_globa_nsbtune['ring_size'][(df_sim_data_globa_nsbtune['ring_radius'] > 1.06) & (df_sim_data_globa_nsbtune['ring_radius'] < 1.08)])}")

In [ ]:
len(df_sim_data_globa_nsbtune['ring_size'][(df_sim_data_globa_nsbtune['ring_radius'] > 1.06) & (df_sim_data_globa_nsbtune['ring_radius'] < 1.08)])

In [ ]:
len(df_real_data['ring_size'][(df_real_data['ring_radius'] > 1.06) & (df_real_data['ring_radius'] < 1.08)])

In [ ]:
plt.hist(df_sim_data_globa_nsbtune['ring_radius'], alpha = 0.5, density=True)
plt.hist(df_real_data_cut['ring_radius'], alpha = 0.5, density=True)

In [ ]:
plt.hist(df_sim_cut['ring_radius'], alpha = 1, density=True, histtype='step', bins = 10)
plt.hist(df_real_cut['ring_radius'], alpha = 1, density=True, histtype='step', bins = 10)

In [ ]:
max(df_sim_cut['ring_radius'])

In [ ]:
max(df_real_cut['ring_radius'])